# Artifact of Table 4 for Phase Moducation Side Channel Analysis and JTAG Attacks

This is part of a series of artifacts. Note these notebooks are designed to run on Google Colab so have been simplified from those available in the [full repository](https://github.com/colinoflynn/phase-modulation-sca).

To run these notebooks, simply run from top to bottom.

**WARNING**: Some steps, such as the extraction, can be very slow. Try not to forget you started the process as CoLab will kill unattended notebooks eventually, and you'll have to start again.

In [ ]:
import sys
!{sys.executable} -m pip install scared

In [ ]:
!sudo apt-get install zstd

In [ ]:
!wget https://zenodo.org/records/13359322/files/jtagpm.tar.zst?download=1

In [ ]:
!tar -xf jtagpm.tar.zst?download=1

In [ ]:
!ls -l pmjtag_cpa

In [ ]:
!rm -rf jtagpm.tar.zst?download=1

In [ ]:
import scared
import estraces
import numpy as np

In [ ]:
knownkey = [43, 126, 21, 22, 40, 174, 210, 166, 171, 247, 21, 136, 9, 207, 79, 60]

In [ ]:
def find_success(att, knownkey):
    recovered_masterkey = np.argmax(att.scores, axis=0).astype('uint8')
    print(recovered_masterkey == knownkey)

    for i in range(0, np.shape(att.convergence_traces)[2]):
        if np.all(np.argmax(att.convergence_traces[:,:,i], axis=0) == knownkey):
            return (i+1)*att.convergence_step
    return -1

In [ ]:
import scared
from scipy import signal


def do_cpa(filename, step=25, window=None, hpf=None, lpf=None):
    ths = estraces.read_ths_from_ets_file(filename)
    print(ths)

    selection_function = scared.aes.selection_functions.encrypt.FirstSubBytes()

    att = scared.CPAAttack(selection_function=selection_function,
                           model=scared.HammingWeight(),
                           discriminant=scared.maxabs,
                           convergence_step=step)

    processes = []

    # Simple high pass filter (HPF)
    if hpf:
        sos = signal.butter(5, hpf, 'highpass', output='sos')

        @scared.preprocess
        def hpfproc(traces):
            return signal.sosfilt(sos, traces)

        processes.append(hpfproc)

    # Simple low pass filter (LPF)
    if lpf:
        sos = signal.butter(5, lpf, 'lowpass', output='sos')

        @scared.preprocess
        def lpfproc(traces):
            return signal.sosfilt(sos, traces)

        processes.append(lpfproc)

    # This resync preprocessing isn't really used, was used with some JTAG captures only
    @scared.preprocess
    def resync(traces):
        import numpy as np
        first = []
        for w in traces:
            first.append(w[0])
        avg = np.mean(first)

        tracesresync = []
        for w in traces:
            if w[0] < avg:
                tracesresync.append(w[:-1])
            else:
                tracesresync.append(w[1:])
        return np.array(tracesresync)
    #processes.append(resync)

    if window:
        container = scared.Container(ths, frame=range(window[0], window[1]), preprocesses=processes)
    else:
        container = scared.Container(ths, preprocesses=processes)


    att.run(container)

    import numpy as np

    recovered_masterkey = np.argmax(att.scores, axis=0).astype('uint8')

    return att

In [ ]:
# Datasets in baseline_traces_cpa online
filenames = [
             ('pmjtag_cpa/cwhusky_shunt_15mhzcpu_30mhzadc_sync.ets', 100, None, None, None),
             ('pmjtag_cpa/cwhusky_jtag_mixer_15mhzcpu_30mhzadc_sync_k.ets', 25, None, None, None), # 8900 @ 25 steps
             ('pmjtag_cpa/cwhusky_jtagclockoff_mixer_15mhzcpu_30mhzadc_sync.ets', 500, None, None, None),
            ]

## Doing all the CPA attacks on JTAG Phase Modulation measurements

The following will do all the attacks on *phase modulation* measurements. The results get saved to a `.npy` file for further use. The `.npy` files have also been saved from prior runs.

In [ ]:
for f in filenames:
    att = do_cpa(f[0], f[1], f[2], f[3], f[4])
    successnum = find_success(att, knownkey)
    print("{:s} = {:d}".format(f[0], successnum))
    np.save(f[0] + ".npy", att)

In [ ]:
%%bash
mkdir results
mv baseline_traces_cpa/*.npy results/.

In [ ]:
!7z a results.7z results

The above results in a bunch of output, but the important parts are the lines that look like this:

```
pmjtag_cpa/cwhusky_shunt_15mhzcpu_30mhzadc_sync.ets = 1800
pmjtag_cpa/cwhusky_jtag_mixer_15mhzcpu_30mhzadc_sync_k.ets = 8900
pmjtag_cpa/cwhusky_jtagclockoff_mixer_15mhzcpu_30mhzadc_sync.ets = -1
```

The above can be found in Table 4, showing the CPA results of the JTAG PM-SCA. We'll recreate the T-Test results below.

Table 4 is recreated here for convience:


| Measurement Basis         | TVLA | CPA  |
|---------------------------|--------|------|
| Shunt on VCC              | 24.0   | 1800 |
| JTAG: Mixer, Toggling TDO | 11.7   | 8900 |
| JTAG: Mixer, Constant TDO | 4.0    | --   |

Note that the final constant TDO does not have a successful result report.


You may wish to save the .npy files. While we have a copy of them, for a "complete" artifact recreation you should copy the .npy files to the CPA plotting notebook.

The easiest way to do this if using colab (and not a local Jupyter install) is to copy them to a directory & compress them. Then download the file in the CoLab web interface.

The following block will move them to a `results` directory, then the block after that will compress that directory. Look for a `results.7z` file you download.

In [ ]:
%%bash
mkdir results
mv baseline_traces_cpa/*.npy results/.

In [ ]:
!7z a results.7z results

## T-Test Results

The following recreats the T-Test results for Table 4.

In [ ]:
import os
import numpy as np
def plot_t(t_val, N, titledata=""):
    import matplotlib.pylab as plt
    ldata = len(t_val)
    plt.plot(t_val)
    plt.plot([0, ldata], [4.5, 4.5], 'k--')
    plt.plot([0, ldata], [-4.5, -4.5], 'k--')
    plt.title("TVLA Results, N=%d, %s"%(N, titledata))
    plt.xlabel("Sample Number")
    plt.ylabel("T-Test Value")

#Set root data dir, not "mixers"
data_dir = ""

def npload(filename):
    return np.load(os.path.join(data_dir, filename), allow_pickle=True)

def nptload(prefix):
    group1 = npload(prefix + "_tvla_group1.npy")
    group2 = npload(prefix + "_tvla_group2.npy")
    N = round((len(group1) + len(group2)) / 2)
    return group1, group2, N

def npsave(filename, array):
    np.save(os.path.join(data_dir, filename), array)

def nptsave(prefix, group1, group2):
    npsave(prefix + "_tvla_group1.npy", group1)
    npsave(prefix + "_tvla_group2.npy", group2)


In [ ]:
files = [
         [r"pmjtag_test/cwhusky_shunt_15mhzcpu_30mhzadc_10ktraces_sync", 100, None, None, None],
         [r"pmjtag_test/cwhusky_jtag_mixer_15mhzcpu_30mhzadc_10ktraces_sync", 100, None, None, None],
         [r"pmjtag_test/cwhusky_jtag_mixer_clockoff_15mhzcpu_30mhzadc_10ktraces_sync", 100, None, None, None],

        ]

In [ ]:
from scipy.stats import ttest_ind
import matplotlib.pylab as plt
from scipy import signal

def find_max_t(file, start=0, end=-1, hpf=None, lpf=None, resync_jtag=False):
    group1, group2, N = nptload(file)

    if resync_jtag:
        first = []
        for g in group1:
            first.append(g[0])
        avg = np.mean(first)
        group1resync = []
        for g in group1:
            if g[0] > avg:
                group1resync.append(g[:-1])
            else:
                group1resync.append(g[1:])

        group2resync = []
        for g in group2:
            if g[0] > avg:
                group2resync.append(g[:-1])
            else:
                group2resync.append(g[1:])

        group1 = group1resync
        group2 = group2resync

    if hpf:
        sos = signal.butter(5, hpf, 'highpass', output='sos')
        group1 = signal.sosfilt(sos, group1)
        group2 = signal.sosfilt(sos, group2)

    if lpf:
        sos = signal.butter(5, lpf, 'lowpass', output='sos')
        group1 = signal.sosfilt(sos, group1)
        group2 = signal.sosfilt(sos, group2)

    t_val = ttest_ind(group1[start:end], group2[start:end], axis=0, equal_var=False)[0]
    max_t = max(abs(t_val))
    return max_t, t_val
for f in files:
    max_t, t_val = find_max_t(f[0], start=f[1], end=f[2], hpf=f[3], lpf=f[4])
    print("{:s}: {:f}".format(f[0], max_t))

This outputs the T-Test results. You can confirm the values match the table above (Table 4 in paper):

```
pmjtag_test/cwhusky_shunt_15mhzcpu_30mhzadc_10ktraces_sync: 23.980417
pmjtag_test/cwhusky_jtag_mixer_15mhzcpu_30mhzadc_10ktraces_sync: 11.718989
pmjtag_test/cwhusky_jtag_mixer_clockoff_15mhzcpu_30mhzadc_10ktraces_sync: 3.643309
```

Note the final value is slightly off. The dataset in the paper used the a slightly different dataset & the one here was recaptued. The T-Test value is lower so it does not impact the results in the paper itself.


This should now be Table 4 fully reproduced, both CPA and T-Test values.